In [304]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver import ActionChains as ac
from selenium.webdriver.common.by import By as by
from selenium.webdriver.support.ui import Select as sel

import os
import copy
import csv
import html
import chromedriver_autoinstaller
import time


In [305]:
URL_INIT = 'https://chunithm-net-eng.com'
URL_LOGGED_IN = 'https://chunithm-net-eng.com/mobile/home/'
URL_LV_BASE = 'https://chunithm-net-eng.com/mobile/record/musicLevel/'
URL_GENRE_BASE = 'https://chunithm-net-eng.com/mobile/ranking/'
DIFFICULTY_LIST = ['basic', 'advanced', 'expert', 'master', 'ultimate']
lvDict = {}

In [306]:
class WebHandler:
    def __init__(self):
        self.DRIVER_PATH = chromedriver_autoinstaller.install()
        self.driver: webdriver.Chrome = None
        
        dir = ['./lv', './genre']
        for x in dir:
            if not os.path.exists(x):
                os.mkdir(x)
    
    def openWeb(self):
        self.driver = webdriver.Chrome(self.DRIVER_PATH)
        self.driver.get(URL_INIT)
        
    def grabHtml(self):
        while self.driver.current_url != URL_LOGGED_IN:
            print(self.driver.current_url)
            time.sleep(0.5)
            
        self.driver.get(URL_LV_BASE)
        with open('./tmp.html', 'w', encoding='UTF-8') as f:
            f.write(self.driver.page_source)
        
        page = open('./tmp.html', 'rt', encoding='utf-8').read()
        lvOptions = bs(page, 'html.parser').find("select").find_all("option")
        
        for x in lvOptions:
            lvDict[x.attrs["value"]] = x.text
        
        for key in lvDict.keys():
            select = sel(self.driver.find_element(by.NAME, "level"))
            select.select_by_value(key)
            
            with open('./lv/' + lvDict[key] + '.html', 'w', encoding='UTF-8') as f:
                f.write(self.driver.page_source)
        
        self.driver.get(URL_GENRE_BASE)
        for x in DIFFICULTY_LIST:
            e = self.driver.find_element(by.CLASS_NAME, "btn_" + x)
            ac(self.driver).click(e).perform()
            
            with open('./genre/' + x + '.html', 'w', encoding='UTF-8') as f:
                f.write(self.driver.page_source)

In [307]:
class Detail:
    def __init__(self):
        self.genre: str = None
        self.lv: list = list()

In [308]:
def compare(x: str, y: str):
    tmpX = x
    tmpY = y
    
    if '+' in tmpX:
        tmpX = x.replace('+', '.5')
    
    if '+' in tmpY:
        tmpY = y.replace('+', '.5')
    
    tmpX = float(tmpX)
    tmpY = float(tmpY)
    
    return 0 if tmpX == tmpY else tmpX < tmpY

In [309]:
def grabData():
    LV_FILES = list()
    GENRE_FILES = list()
    music: dict[str, Detail] = {}

    for x in os.listdir('./lv'):
        if '.html' in x:
            LV_FILES.append(x)

    for x in os.listdir('./genre'):
        if '.html' in x:
            GENRE_FILES.append(x)
            
    for f in LV_FILES:
        page = open('./lv/' + f, 'rt', encoding='utf-8').read()
        soup = bs(page, 'html.parser')
        lv = f.replace('lv', '').replace('.html', '')
        
        for x in soup.find_all('div', 'music_title'):     
            name = x.text     
            if not name in music:
                music[name] = Detail()
            music[name].lv.append(lv)

    for f in GENRE_FILES:
        page = open('./genre/' + f, 'rt', encoding='utf-8').read()
        soup = bs(page, 'html.parser')
        
        currentGenre = None
        for line in str(soup.prettify).split('\n'):
            if '<div class="genre scroll_point text_white">' in line:
                currentGenre = html.unescape(line[line.index('>')+1:line.index('<', 2)])
                
            if '<div class="music_title">' in line:
                name = html.unescape(line[line.index('>')+1:line.index('<', 2)])
                music[name].genre = currentGenre
      
    with open('output.csv', 'w', encoding='UTF-8') as f:
        for key in music.keys():
            f.write("%s, %s, " % (key, music[key].genre))
            
            for lv in music[key].lv:
                lv = lv.replace("LEVEL ", "")
                f.write("%s, " % (lv) )
            f.write("\n")

In [310]:
if __name__ == "__main__":
    wh = WebHandler()
    wh.openWeb()
    wh.grabHtml()
    
    grabData()

C:\Users\Toygoon\AppData\Local\Temp\ipykernel_18680\2090486308.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(self.DRIVER_PATH)


https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&redirect_url=https://chunithm-net-eng.com/mobile/&back_url=https://chunithm.sega.com/
https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&redirect_url=https://chunithm-net-eng.com/mobile/&back_url=https://chunithm.sega.com/
https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&redirect_url=https://chunithm-net-eng.com/mobile/&back_url=https://chunithm.sega.com/
https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&redirect_url=https://chunithm-net-eng.com/mobile/&back_url=https://chunithm.sega.com/
https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&redirect_url=https://chunithm-net-eng.com/mobile/&back_url=https://chunithm.sega.com/
https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&redirect_url=https://chunithm-net-eng.com/mobile/&back_url=https://chunithm.sega.com/
https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&r

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'